Ön hazırlık (kütüphane kurulumu)

In [1]:
# Gerekli kütüphaneleri yükleyelim (Colab hücresine)
!pip install cirq qiskit qutip numpy matplotlib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.8/670.8 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 430.5/430.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.8/31.8 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 75.7 MB/s eta 0:00:00


LİNEER CEBİR VE MATRİS İŞLEMLERİ

5.1. HARROW-HASSIDIM-LLOYD ALGORİTMASI (HHL)

Qiskit — toy HHL (2×2, QPE via controlled-Unitary)

Gereksinimler: qiskit>=0.25 (Aer simülatörü), numpy, scipy (expm)

In [3]:
# qiskit_hhl_toy.py
import numpy as np
from scipy.linalg import expm, eig
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.extensions import UnitaryGate
from qiskit.circuit.library import QFT
from qiskit.visualization import plot_histogram

# --- Problem setup (toy A and |b>)
A = np.array([[1.0, 0.0],
              [0.0, 2.0]])
# eigen-decomposition (diagonal already)
eigvals, eigvecs = eig(A)
print("eigvals:", eigvals)

# choose U = exp(2π i A / 4)
U = expm(2j * np.pi * A / 4)

# parameters
n_count = 3  # number of counting (QPE) qubits
n_sys = 1    # system qubits
n_anc = 1    # ancilla for controlled rotation

# Build circuit: [count register] [system qubit] [ancilla]
qc = QuantumCircuit(n_count + n_sys + n_anc, n_count)

# 1) Prepare |b> on system. Here |b>=|0>, so do nothing. If different, use initialize.
# For demonstration, prepare |b> = |0>
sys_index = n_count
anc_index = n_count + n_sys

# 2) Hadamard on counting qubits
for q in range(n_count):
    qc.h(q)

# 3) QPE: apply controlled-U^{2^j} from counting qubit j to system qubit
# build UnitaryGate from U
# We need controlled-U^{2^j}, use UnitaryGate and .control()
for j in range(n_count):
    power = 2**j
    Uj = np.linalg.matrix_power(U, power)
    gate = UnitaryGate(Uj)
    # make controlled and attach: control on qubit j, target sys_index
    cgate = gate.control(1)  # 1 control
    # cgate acts on [control, target]
    qc.append(cgate, [j, sys_index])

# 4) inverse QFT on counting register
qc.append(QFT(num_qubits=n_count, do_swaps=True).inverse(), range(n_count))

# 5) controlled rotation on ancilla conditioned on counting register states.
# For our toy problem, after inverse QFT the counting register basis states approximately correspond to
# |001> -> eigenvalue 1/4, |010> etc. We'll implement controlled rotations by enumerating
# the 2^n_count basis: if basis corresponds to lambda_j we rotate ancilla by theta = 2*arcsin(C/lambda_j)
# (for small C approx C/lambda). We use a multi-controlled single-qubit rotation by constructing
# a simple implementation: decompose control by flipping states into a single control via X and mcx,
# but for simplicity in this toy code we condition on measurement later (post-selection).
#
# Simpler educational approach: do NOT apply multi-controlled rotation in circuit; instead we will
# extract counting register probabilities, infer eigenvalue and classically construct the final |x>.
#
qc.barrier()

# measurement of counting register (we will simulate and read distribution)
qc.measure(range(n_count), range(n_count))

# Run
backend = Aer.get_backend('aer_simulator')
tqc = transpile(qc, backend)
result = backend.run(tqc, shots=1024).result()
counts = result.get_counts()
print("Counting register counts:", counts)

# Interpretation (educational):
# If we get measurement result close to 0.010 (binary) = decimal 2 => corresponds to phi=1/4 eigenvalue
# or 0.100 (binary) = decimal 4 => corresponds to phi=1/2 eigenvalue.
# After obtaining eigenvalue (or a sample), classically apply 1/λ factor to compute x = A^{-1} b.
#
# For this toy A and b=|0>, solution x is proportional to A^{-1}|b> = [1/1, 0] = |0> (so trivial)
#
# Display histogram (optional)
try:
    from matplotlib import pyplot as plt
    plot_histogram(counts)
    plt.show()
except Exception:
    pass


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

Cirq — toy HHL (aynı fikir, controlled MatrixGate ile QPE)

In [4]:
# cirq_hhl_toy.py
import numpy as np
import cirq
from scipy.linalg import expm

# Problem setup (same A)
A = np.array([[1.0, 0.0],
              [0.0, 2.0]])
U = expm(2j * np.pi * A / 4)

n_count = 3
controls = [cirq.LineQubit(i) for i in range(n_count)]
sys_q = cirq.LineQubit(n_count)
anc = cirq.LineQubit(n_count+1)

circuit = cirq.Circuit()

# prepare system |b>=|0> -> nothing; if needed use X or initializer
# Hadamards on counting qubits
circuit.append([cirq.H(q) for q in controls])

# Controlled-U^(2^j) : Cirq allows controlled gates from MatrixGate
for j, q in enumerate(controls):
    power = 2**j
    Uj = np.linalg.matrix_power(U, power)
    # construct gate acting on system qubit
    mg = cirq.MatrixGate(Uj)
    # controlled version
    cgate = mg.controlled()
    circuit.append(cgate(q, sys_q))

# inverse QFT (implement simple inverse QFT function)
def inverse_qft(qubits):
    ops = []
    n = len(qubits)
    # swap
    for i in range(n//2):
        ops.append(cirq.SWAP(qubits[i], qubits[n-i-1]))
    # inverse rotations + H
    for j in range(n):
        for k in range(j):
            ops.append(cirq.CZ(qubits[j], qubits[k])**(-1/2**(j-k)))
        ops.append(cirq.H(qubits[j]))
    return ops

circuit.append(inverse_qft(controls))

# measure counting register
circuit.append([cirq.measure(q, key=f'c{idx}') for idx,q in enumerate(controls)])

sim = cirq.Simulator()
res = sim.run(circuit, repetitions=1000)
print(res.histogram(keys=[f'c{i}' for i in range(n_count)]))
print(circuit)


TypeError: Result.histogram() got an unexpected keyword argument 'keys'

QuTiP — tam toy HHL simülasyonu (matris tabanlı, tam statevector kullanımı)

In [5]:
# qutip_hhl_toy.py
import numpy as np
from scipy.linalg import expm
from qutip import tensor, basis, qeye, Qobj, ket2dm

# --- Problem definition: A (2x2), |b>
A = np.array([[1.0, 0.0],
              [0.0, 2.0]])
# eigen-decomposition
eigvals, eigvecs = np.linalg.eigh(A)
print("Eigenvalues:", eigvals)

# choose U = exp(2π i A / 4) so phases = lambda / 4
U_sys = expm(2j * np.pi * A / 4)  # 2x2

# register sizes
n_count = 3   # QPE qubits
n_sys = 1     # system qubit
n_anc = 1     # ancilla for controlled rotation
N_count = 2**n_count

# build full Hilbert space operators using Qobj
from qutip import Qobj
U_sys_q = Qobj(U_sys)

# basis states
zero = basis(2,0)
one  = basis(2,1)

# Initial total state |0>^{count} ⊗ |b> ⊗ |0>_anc
# here |b>=|0>
count_zeros = tensor([zero for _ in range(n_count)])
sys_b = zero
anc0 = zero
psi0 = tensor(count_zeros, sys_b, anc0)

# 1) Apply Hadamard to each counting qubit: build H_full
H = (1/np.sqrt(2)) * np.array([[1,1],[1,-1]])
H_q = Qobj(H)
H_counts_full = tensor(*([H_q]*n_count), qeye(2), qeye(2))  # counts x sys x anc
psi1 = H_counts_full * psi0

# 2) Apply controlled-U^{2^j}: build full unitary for each j
# We'll compose the full operator as product of controlled operators
def controlled_unitary_on_sys(U_mat, control_index, n_count, n_sys, n_anc):
    # control_index in [0..n_count-1], 0 is first qubit
    # Build projector P0 and P1 on that control qubit
    P0 = Qobj([[1,0],[0,0]])
    P1 = Qobj([[0,0],[0,1]])
    # build identity lists
    ops_P0 = []
    ops_P1 = []
    for i in range(n_count):
        if i == control_index:
            ops_P0.append(P0)
            ops_P1.append(P1)
        else:
            ops_P0.append(qeye(2))
            ops_P1.append(qeye(2))
    # append system and anc spaces
    ops_P0.extend([qeye(2)] * (n_sys + n_anc))
    ops_P1.extend([Qobj(U_mat)] + [qeye(2)] * n_anc)  # U acts on system only
    Op = tensor(*ops_P0) + tensor(*ops_P1)
    return Op

U_total = Qobj(np.eye(2**(n_count+n_sys+n_anc)), dims=[[2]*(n_count+n_sys+n_anc), [2]*(n_count+n_sys+n_anc)])
psi2 = psi1
for j in range(n_count):
    power = 2**j
    Uj = np.linalg.matrix_power(U_sys, power)
    C_Uj = controlled_unitary_on_sys(Uj, j, n_count, n_sys, n_anc)
    psi2 = C_Uj * psi2

# 3) Apply inverse QFT on counting register (as unitary)
# build QFT matrix on n_count qubits
def qft_matrix(n):
    N = 2**n
    omega = np.exp(2j*np.pi / N)
    F = np.zeros((N,N), dtype=complex)
    for x in range(N):
        for y in range(N):
            F[x,y] = omega**(x*y)
    return F/np.sqrt(N)

QFT_n = Qobj(qft_matrix(n_count))
# embed QFT^\dagger on counting register
QFT_dagger_full = tensor(QFT_n.dag(), qeye(2), qeye(2))
psi3 = QFT_dagger_full * psi2

# 4) Controlled rotation on ancilla depending on counting register value
# After QFT_dagger, counting register computational basis approx encodes eigenvalue index.
# We'll implement controlled rotations R_y(θ_k) where θ_k = 2*arcsin(C/λ_k).
# For our toy eigenvalues λ∈{1,2}, choose mapping from counting basis states to nearest eigenvalue.
# Map: if counting register state corresponds to decimal ~2 -> eigenvalue 1; decimal ~4 -> eigenvalue 2.
# We'll implement control by projector onto that counting basis state.
C = 1.0
N_count = 2**n_count
psi4 = psi3
for basis_idx in range(N_count):
    # decide which eigenvalue this basis_idx corresponds to (simple nearest test)
    # get phase estimate phi = basis_idx / 2^n_count
    phi_est = basis_idx / float(N_count)
    # nearest eigenvalue ratio is phi_est*4 approx lambda
    lambda_est = round(phi_est * 4)
    if lambda_est == 0:
        continue
    theta = 2 * np.arcsin(C / lambda_est) if abs(C/lambda_est) <= 1 else np.pi
    # projector onto |basis_idx> on counting register
    ket_idx = basis(N_count, basis_idx)
    P_idx = ket_idx * ket_idx.dag()
    # expand to full space: P_idx ⊗ I_sys ⊗ Ry(θ)_anc
    from qutip import rx, ry, rz
    Ry = Qobj(expm(-1j * theta/2 * np.array([[0,-1j],[1j,0]])))  # explicit Ry
    full_op = tensor(P_idx, qeye(2), Ry) + tensor((qeye(N_count)-P_idx), qeye(2), qeye(2))
    psi4 = full_op * psi4

# 5) Uncompute QFT & uncompute QPE: for small toy we'll skip explicit uncompute and instead
# trace out counting register, postselect ancilla=|1>, and get resulting system state.
# (In full HHL you'd uncompute counting register to disentangle)
from qutip import measurement_statistics_povm
# get reduced density on ancilla and system
rho_full = ket2dm(psi4)
# trace over counting register to obtain joint (sys,anc)
# dims: [count]*(n_count) , [sys], [anc]
rho_sysanc = rho_full.ptrace(list(range(n_count, n_count + n_sys + n_anc)))
# measure ancilla in computational basis and postselect on |1>
proj_anc_1 = tensor(qeye(2), basis(2,1)*basis(2,1).dag())  # sys x anc
# actually need operators on sys+anc: project anc
P_anc1 = tensor(qeye(2), basis(2,1)*basis(2,1).dag())
# probability of anc=1
prob_anc1 = (P_anc1 * rho_sysanc).tr().real
print("Prob(ancilla=1):", prob_anc1)

if prob_anc1 > 1e-9:
    # postselected state (sys only)
    rho_post = (P_anc1 * rho_sysanc * P_anc1) / prob_anc1
    # trace out anc to get system density
    rho_sys = rho_post.ptrace(0)
    # compute fidelity with ideal |x> ∝ A^{-1} |b>
    x_exact = np.linalg.inv(A) @ np.array([1.0, 0.0])
    x_exact = x_exact / np.linalg.norm(x_exact)
    from qutip import Qobj
    psi_x_exact = Qobj(x_exact)
    fidelity = (psi_x_exact.dag() * rho_sys * psi_x_exact).tr().real
    print("Fidelity with exact solution |x>:", fidelity)
else:
    print("Postselection failed (ancilla=1 prob too small).")


Eigenvalues: [1. 2.]


TypeError: incompatible dimensions [[8, 2, 2], [8, 2, 2]] and [[2, 2, 2, 2, 2], [1]]

QUANTUM SİNGULAR VALUE TRANSFORMATİON
(QSVT)

Qiskit — Toy QSVT (1 ancilla qubit + 1 system qubit)

In [6]:
# qiskit_qsvt_toy.py
# Gerekli paketler: qiskit, numpy, scipy
import numpy as np
from scipy.linalg import sqrtm
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.extensions import UnitaryGate

# ---------------------------
# 1) Problem: küçük diag(A) örneği
# ---------------------------
# Tekil değerler (sigma'lar) 0..1 aralığında
s0 = 0.6
s1 = 0.9
# A = diag(s0, s1)
A = np.diag([s0, s1])

# normalizasyon katsayısı (alpha) = 1 (sigma <=1 olduğu için)
alpha = 1.0

# ---------------------------
# 2) Blok-kodlama: 4x4 unitary oluştur
#    top-left = A, top-right = sqrt(I - A^2)
#    bottom-left = sqrt(I - A^2), bottom-right = -A
#    Bu, her tekil değer için 2x2 rotasyon bloğu oluşturur => 4x4 unitary
# ---------------------------
I2 = np.eye(2)
sqrt_term = sqrtm(I2 - A.dot(A))  # sqrt(I - A^2)
U_block = np.block([[A, sqrt_term],
                    [sqrt_term, -A]])
# kontrol: U_block birimleriyle unitary mi?
print("U_block^† U_block hata (küçük olmalı):", np.linalg.norm(U_block.conj().T @ U_block - np.eye(4)))

# ---------------------------
# 3) QSVT tarzı "alternating" sequence (öğretici)
#    Sıra: (apply U) -> Rz(φ1) on ancilla -> (apply U†) -> Rz(φ2) -> (apply U)
#    (bu basit sequence mamafih tam QSVT polinom tasarımı değildir, ama mantığı gösterir)
# ---------------------------
# örnek fazlar: küçük dizi (örnek)
phi_list = [np.pi/2, np.pi/3]  # ancilla üzerindeki Rz fazları (öğretici amaçlı)

# Qiskit devre: 2 qubit (ancilla: q0, system: q1)
qc = QuantumCircuit(2)
# devrede matrisler Qiskit sütun sırasına göre uygulanır (q0 is ancilla)
# apply U_block as a 2-qubit unitary
U_gate = UnitaryGate(U_block, label='U_block')
# 1) U
qc.append(U_gate, [0,1])
# 2) Rz(phi1) on ancilla
qc.rz(phi_list[0], 0)
# 3) U†
qc.append(UnitaryGate(U_block.conj().T, label='U_dag'), [0,1])
# 4) Rz(phi2)
qc.rz(phi_list[1], 0)
# 5) U
qc.append(U_gate, [0,1])

# 6) ölçümlü değil — statevector üzerinden sonuç çıkaracağız
backend = Aer.get_backend('aer_simulator_statevector')
tqc = transpile(qc, backend)
result = backend.run(tqc).result()
psi = result.get_statevector()  # 4 amplitüdlü statevector (ancilla ⊗ system)

# ---------------------------
# 4) Post-select ancilla = |0> ve sistem üzerine etki eden effective operatörü çıkar
#    (başlangıç sistem baz vektörlerini uygulayıp mapping M : |ψ> -> <0| (QSVTsequence) |0>⊗|ψ> )
# ---------------------------
# ilk olarak baz vektörleri için (|0>, |1>) test et
zero_sys = np.array([1,0], dtype=complex)
one_sys  = np.array([0,1], dtype=complex)

# fonksiyon: full unitary U_seq olarak uygulayıp ancilla projeksiyonuyla sistem haritası elde et
from numpy import kron

# compute full unitary matrix of the circuit (manually build sequence)
U = U_block
U_dag = U_block.conj().T
# full sequence: U * (Rz on ancilla) * U_dag * (Rz on ancilla) * U
Rz_phi1 = np.kron(np.array([[np.exp(-1j*phi_list[0]/2), 0],
                             [0, np.exp(1j*phi_list[0]/2)]]), np.eye(2))
Rz_phi2 = np.kron(np.array([[np.exp(-1j*phi_list[1]/2), 0],
                             [0, np.exp(1j*phi_list[1]/2)]]), np.eye(2))_*]()


SyntaxError: unmatched ']' (ipython-input-758288482.py, line 81)

Cirq — aynı toy QSVT akışı (Cirq MatrixGate kullanılarak)

In [7]:
# cirq_qsvt_toy.py
# Gerekli paketler: cirq, numpy, scipy
import cirq
import numpy as np
from scipy.linalg import sqrtm

# ---------------------------
# 1) Problem: A = diag(s0, s1)
# ---------------------------
s0 = 0.6
s1 = 0.9
A = np.diag([s0, s1])
I2 = np.eye(2)
sqrt_term = sqrtm(I2 - A.dot(A))
U_block = np.block([[A, sqrt_term],
                    [sqrt_term, -A]])
# birimlik kontrol
print("U_block unitary check (Cirq):", np.linalg.norm(U_block.conj().T @ U_block - np.eye(4)))

# ---------------------------
# 2) QSVT alternation sequence (öğretici)
# ---------------------------
phi_list = [np.pi/2, np.pi/3]

# qubit tanımları: ancilla q0, system q1
q0 = cirq.LineQubit(0)  # ancilla
q1 = cirq.LineQubit(1)  # system

circuit = cirq.Circuit()

# 1) U
circuit.append(cirq.MatrixGate(U_block).on(q0, q1))
# 2) Rz(phi1) on ancilla
circuit.append(cirq.rz(phi_list[0]).on(q0))
# 3) U†
circuit.append(cirq.MatrixGate(U_block.conj().T).on(q0, q1))
# 4) Rz(phi2)
circuit.append(cirq.rz(phi_list[1]).on(q0))
# 5) U
circuit.append(cirq.MatrixGate(U_block).on(q0, q1))

# Circuit gösterimi
print("Cirq devresi:\n", circuit)

# ---------------------------
# 3) Simülasyon: statevector üzerinden etkileri hesapla
# ---------------------------
sim = cirq.Simulator()
# başlangıç sistem baz vektörleri için M matrisini elde etmek üzere her baz için simüle et
basis_sys = [np.array([1,0], dtype=complex), np.array([0,1], dtype=complex)]
M = np.zeros((2,2), dtype=complex)
for j, b in enumerate(basis_sys):
    init = np.kron(np.array([1,0]), b)  # ancilla |0> ⊗ system |b>
    # simülasyon: Cirq expect statevector from unitary sequence by applying unitary matrix product
    # kolaylık: hesapları doğrudan numpy ile yap (aynı U_seq mantığı)
    # construct Rz matrices as kron(Rz, I)
    Rz1 = np.kron(np.array([[np.exp(-1j*phi_list[0]/2),0],[0,np.exp(1j*phi_list[0]/2)]]), np.eye(2))
    Rz2 = np.kron(np.array([[np.exp(-1j*phi_list[1]/2),0],[0,np.exp(1j*phi_list[1]/2)]]), np.eye(2))
    U = U_block
    U_dag = U_block.conj().T
    U_seq = U @ Rz1 @ U_dag @ Rz2 @ U
    out = U_seq.dot(init)
    # ancilla=|0> için komponentler out[0], out[1] (ancilla index 0)
    sys_vec = np.array([out[0], out[1]])
    M[:, j] = sys_vec

print("Cirq ile elde edilen M (unnormalized):\n", M)

# Klasik referans f(A)=A^2 örnek
fA = n


U_block unitary check (Cirq): 4.3403273643888066e-17
Cirq devresi:
       ┌                           ┐              ┌                           ┐                ┌                           ┐
      │ 0.6    0.     0.8    0.   │              │ 0.6    0.     0.8    0.   │                │ 0.6    0.     0.8    0.   │
0: ───│ 0.     0.9    0.     0.436│───Rz(0.5π)───│ 0.     0.9    0.     0.436│───Rz(0.333π)───│ 0.     0.9    0.     0.436│───
      │ 0.8    0.    -0.6   -0.   │              │ 0.8    0.    -0.6   -0.   │                │ 0.8    0.    -0.6   -0.   │
      │ 0.     0.436 -0.    -0.9  │              │ 0.     0.436 -0.    -0.9  │                │ 0.     0.436 -0.    -0.9  │
      └                           ┘              └                           ┘                └                           ┘
      │                                          │                                            │
1: ───#2─────────────────────────────────────────#2──────────────────────────────────────

NameError: name 'n' is not defined

QuTiP — Matris tabanlı tam HILBERT örneği (en net gösterim)

In [8]:
# qutip_qsvt_toy.py
# Gerekli paketler: qutip, numpy, scipy
import numpy as np
from scipy.linalg import sqrtm
from qutip import Qobj, tensor, basis, ket2dm

# ---------------------------
# 1) Problem: A = diag(s0, s1)
# ---------------------------
s0 = 0.6
s1 = 0.9
A = np.diag([s0, s1])
I2 = np.eye(2)
sqrt_term = sqrtm(I2 - A.dot(A))
# 4x4 block unitary
U_block = np.block([[A, sqrt_term],
                    [sqrt_term, -A]])
# doğruluk kontrolü
print("U_block unitary check (QuTiP):", np.linalg.norm(U_block.conj().T @ U_block - np.eye(4)))

# Qobj formuna çevir
U_q = Qobj(U_block)

# ---------------------------
# 2) QSVT alternation sequence
# ---------------------------
phi_list = [np.pi/2, np.pi/3]

# Rz(φ) on ancilla as Qobj
def Rz_qobj(phi):
    Rz = np.array([[np.exp(-1j*phi/2), 0],
                   [0, np.exp(1j*phi/2)]], dtype=complex)
    return Qobj(Rz)

# full Rz on ancilla = Rz ⊗ I_sys
Rz1_full = tensor(Rz_qobj(phi_list[0]), Qobj(np.eye(2)))
Rz2_full = tensor(Rz_qobj(phi_list[1]), Qobj(np.eye(2)))

# U and U† as Qobj act on ancilla⊗sys subspace
U_qobj = Qobj(U_block)
U_dag_qobj = U_qobj.dag()

# Compose full sequence U_seq = U * Rz1 * U† * Rz2 * U
U_seq = U_qobj * Rz1_full * U_dag_qobj * Rz2_full * U_qobj

# ---------------------------
# 3) Elde edilen M matrisini çıkart (ancilla=|0> projeksiyonu sonrası)
# ---------------------------
# başlangıç vektörleri sistem için |0>, |1>
zero = basis(2,0)
one  = basis(2,1)
M = np.zeros((2,2), dtype=complex)
for j, psi_sys in enumerate([zero, one]):
    psi_in = tensor(basis(2,0), psi_sys)  # ancilla |0> ⊗ system |psi_sys>
    out = U_seq * psi_in
    # ancilla=|0> durumunun komponentleri: projeyle al
    P0 = tensor(basis(2,0)*basis(2,0).dag(), Qobj(np.eye(2)))
    out0 = P0 * out
    # sistem durumunu al (ancilla=0 için full vector entry 0..1)
    # rep: out.full() gives 4 amplitude vector; ancilla=0 components are indices 0,1
    amp = out.full().flatten()
    sys_vec = np.array([amp[0,0], amp[1,0]])
    M[:, j] = sys_vec

print("QuTiP ile M (unnormalized):\n", M)

# Klasik referans f(A) örnek (f(x)=x^2)
fA = np.diag([s0**2, s1**2])
print("Klasik f(A) (örnek f(x)=x^2):\n", fA)

# Postselection olasılığı (ör: başlangıç |ψ> = |0> superpozisyonu ile)
psi_in = tensor(basis(2,0), basis(2,0))
out_global = U_seq * psi_in
rho = ket2dm(out_global)
P_anc1 = tensor(Qobj(np.eye(2)), basis(2,1)*basis(2,1).dag())
prob_anc1 = (P_anc1 * rho).tr().real
print("Örnek: ancilla=|1> prob (örnek) =", prob_anc1)

print("\nNot: QuTiP örneği tüm adımları matris tabanlı olarak gösterir ve postselect ile M'yi çıkarır.")


U_block unitary check (QuTiP): 4.3403273643888066e-17


TypeError: incompatible dimensions [[4], [4]] and [[2, 2], [2, 2]]

QUANTUM PRINCIPAL COMPONENT ANALYSİS
(QPCA)

QPCA — Cirq ile Uygulama

In [9]:
import cirq
import numpy as np

# --- 1. Kübitlerin tanımlanması ---
qubits = [cirq.NamedQubit("q0"), cirq.NamedQubit("q1")]

# --- 2. Veri kümesinden türetilmiş örnek yoğunluk matrisi ---
rho = np.array([[0.7, 0.3],
                [0.3, 0.3]])

# --- 3. Yoğunluk matrisinin özdeğer ve özvektör ayrışımı ---
eigvals, eigvecs = np.linalg.eigh(rho)

print("Özdeğerler:", eigvals)
print("Özvektörler:\n", eigvecs)

# --- 4. En büyük özdeğere karşılık gelen özvektör seçilir ---
principal_component = eigvecs[:, np.argmax(eigvals)]

# --- 5. Bu vektörü kuantum durumuna dönüştür ---
norm_state = principal_component / np.linalg.norm(principal_component)
qc = cirq.Circuit()
qc.append(cirq.StatePreparationChannel(norm_state).on(qubits[0]))

print("\n--- QPCA CIRCUIT ---")
print(qc)

# --- 6. Simülasyon ---
sim = cirq.Simulator()
result = sim.simulate(qc)
print("\nÇıktı durumu:", np.round(result.final_state_vector, 3))


Özdeğerler: [0.13944487 0.86055513]
Özvektörler:
 [[ 0.47185793 -0.8816746 ]
 [-0.8816746  -0.47185793]]

--- QPCA CIRCUIT ---
q0: ───StatePreparation───

Çıktı durumu: [-0.882+0.j -0.472+0.j]


2. QPCA — Qiskit ile Uygulama

In [10]:
from qiskit import QuantumCircuit, Aer, transpile, assemble
from qiskit.quantum_info import DensityMatrix
import numpy as np

# --- 1. Kovaryans matrisi örneği ---
rho = np.array([[0.6, 0.4],
                [0.4, 0.4]])

# --- 2. Yoğunluk matrisi oluşturma ---
dm = DensityMatrix(rho)
eigvals, eigvecs = np.linalg.eigh(rho)

print("Özdeğerler:", eigvals)
print("Özvektörler:\n", eigvecs)

# --- 3. En büyük özdeğere karşılık gelen özvektör seçimi ---
principal = eigvecs[:, np.argmax(eigvals)]

# --- 4. Kuantum devresi oluştur ---
qc = QuantumCircuit(1)
qc.initialize(principal, [0])

print("\n--- QPCA QISKIT DEVRESİ ---")
print(qc)

# --- 5. Simülasyon ---
backend = Aer.get_backend('statevector_simulator')
tqc = transpile(qc, backend)
qobj = assemble(tqc)
result = backend.run(qobj).result()
state = result.get_statevector()
print("\nÇıktı Durumu:", np.round(state, 3))


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

3. QPCA — QuTiP ile Uygulama

In [11]:
from qutip import Qobj, ket2dm, basis
import numpy as np

# --- 1. Kovaryans matrisini (yoğunluk matrisi olarak) tanımla ---
rho_matrix = np.array([[0.6, 0.4],
                       [0.4, 0.4]])

rho = Qobj(rho_matrix)

# --- 2. Özdeğer ve özvektör analizi ---
eigvals, eigvecs = rho.eigenstates()

print("Özdeğerler:", [v for v in eigvals])

# --- 3. En büyük özdeğere sahip bileşen seçilir ---
principal_component = eigvecs[0]  # En büyük özdeğer sıralamada ilk sırada

# --- 4. Çıktı durumunu (principal component) yazdır ---
print("\nAna Bileşen (kuantum durumu):")
print(principal_component)

# --- 5. Yoğunluk matrisi biçiminde gösterim ---
rho_reduced = ket2dm(principal_component)
print("\nAzaltılmış yoğunluk matrisi:")
print(rho_reduced)


Özdeğerler: [np.float64(0.08768943743823393), np.float64(0.912310562561766)]

Ana Bileşen (kuantum durumu):
Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.61541221]
 [-0.78820544]]

Azaltılmış yoğunluk matrisi:
Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.37873219 -0.48507125]
 [-0.48507125  0.62126781]]


QUANTUM LINEAR SYSTEMS ALGORİTHM(QLSA)

1. QLSA — Qiskit ile (HHL benzeri)

In [12]:
from qiskit import Aer, transpile, assemble
from qiskit.circuit.library import HHL
from qiskit.quantum_info import Statevector
import numpy as np

# --- 1. Lineer sistem tanımı ---
# A * x = b
# A 2x2 Hermitian matris, b normalize edilmiş vektör
A = np.array([[1, 1/2],
              [1/2, 1]])
b = np.array([1, 0])
b = b / np.linalg.norm(b)  # normalize

# --- 2. Qiskit'in hazır HHL devresini oluştur ---
num_qubits = 2
hhl = HHL()
circuit = hhl.construct_circuit(matrix=A, vector=b)

print("\n--- QLSA (HHL) Qiskit Devresi ---")
print(circuit.draw())

# --- 3. Simülasyon ---
backend = Aer.get_backend("statevector_simulator")
tqc = transpile(circuit, backend)
qobj = assemble(tqc)
result = backend.run(qobj).result()
state = result.get_statevector()

print("\nÇözümün kuantum temsili (|x>):")
print(np.round(state, 3))


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

QLSA — Cirq ile Temel Uygulama

In [13]:
import cirq
import numpy as np

# --- 1. Kübitleri tanımla ---
q0, q1, ancilla = cirq.LineQubit.range(3)

# --- 2. Basit bir sistem: A * x = b ---
A = np.array([[1, 0.5], [0.5, 1]])
b = np.array([1, 0])
b = b / np.linalg.norm(b)

# --- 3. |b> durumunu hazırla ---
prep = cirq.Circuit(cirq.StatePreparationChannel(b).on(q0))

# --- 4. QPE ve kontrollü rotasyon adımlarını basitleştirilmiş şekilde temsil et ---
# (Gerçek HHL kadar ayrıntılı değil; konsepti gösterir)
circuit = cirq.Circuit()
circuit.append(prep)
circuit.append(cirq.H(ancilla))
circuit.append(cirq.CNOT(q0, ancilla))
circuit.append(cirq.H(ancilla))
circuit.append(cirq.measure(ancilla, key='ancilla'))

print("\n--- QLSA (HHL Basitleştirilmiş) Cirq Devresi ---")
print(circuit)

# --- 5. Simülasyon ---
sim = cirq.Simulator()
result = sim.simulate(circuit)
print("\nFinal durum vektörü:", np.round(result.final_state_vector, 3))



--- QLSA (HHL Basitleştirilmiş) Cirq Devresi ---
0: ───StatePreparation───@──────────────────────
                         │
2: ───H──────────────────X───H───M('ancilla')───

Final durum vektörü: [1.+0.j 0.+0.j 0.+0.j 0.+0.j]


QLSA — QuTiP ile Doğrudan Çözümleme

In [14]:
from qutip import Qobj, ket2dm, basis
import numpy as np

# --- 1. Lineer sistem tanımı ---
A = np.array([[1, 0.5],
              [0.5, 1]])
b = np.array([1, 0])
b = b / np.linalg.norm(b)

# --- 2. Klasik çözümü kuantum biçimde göster ---
x = np.linalg.solve(A, b)
x = x / np.linalg.norm(x)

# --- 3. Kuantum durumu oluştur ---
x_ket = Qobj(x).unit()
rho_x = ket2dm(x_ket)

print("\n--- QLSA (QuTiP) ---")
print("Kuantum durumu |x> =\n", x_ket)
print("\nYoğunluk matrisi:\n", rho_x)

# --- 4. A'nın operatör formu ---
A_qobj = Qobj(A)
# --- 5. Doğrulama: A|x> ≈ |b> ? ---
b_pred = A_qobj * x_ket
print("\nA|x> sonucu (yaklaşık |b>):\n", b_pred)



--- QLSA (QuTiP) ---
Kuantum durumu |x> =
 Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[ 0.89442719]
 [-0.4472136 ]]

Yoğunluk matrisi:
 Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.8 -0.4]
 [-0.4  0.2]]

A|x> sonucu (yaklaşık |b>):
 Quantum object: dims=[[2], [1]], shape=(2, 1), type='ket', dtype=Dense
Qobj data =
[[0.67082039]
 [0.        ]]


HHL Yaklaşımı

QuTiP ile HHL (QLSA) Basit Örneği

In [15]:
import numpy as np
from qutip import tensor, qeye, basis, Qobj, cnot, rx, ry, rz, qeye

# 1. Adım: Sistemin Tanımı
# A * x = b
A = np.array([[1, 0.5],
              [0.5, 1]])
b = np.array([1, 0])

# |b> durumunu normalize et
b_norm = b / np.linalg.norm(b)
ket_b = Qobj(b_norm)

# 2. Adım: A matrisinin özdeğerlerini ve özvektörlerini bul
eigvals, eigvecs = np.linalg.eigh(A)

# Özdeğerleri ve özvektörleri kuantum temsiline dönüştür
U = Qobj(eigvecs)
Lambda = np.diag(eigvals)

# 3. Adım: Quantum Phase Estimation (QPE) benzetimi
# Burada QPE işlemini doğrudan simüle ediyoruz
lambda_inv = np.diag(1 / eigvals)
A_inv = eigvecs @ lambda_inv @ eigvecs.T

# 4. Adım: Kuantum durumda çözümü elde et
x_quantum = Qobj(A_inv @ b_norm)
x_quantum = x_quantum.unit()  # normalize et

print("A:")
print(A)
print("\n|b> =", b_norm)
print("\nA^-1 * |b> (|x>):")
print(x_quantum.full())

# 5. Adım: Ölçüm simülasyonu
probabilities = np.abs(x_quantum.full())**2
print("\nÖlçüm olasılıkları:")
print(probabilities)


ImportError: cannot import name 'cnot' from 'qutip' (/usr/local/lib/python3.12/dist-packages/qutip/__init__.py)

QuTiP ile Kavramsal HHL Simülasyonu

In [16]:
import numpy as np
from qutip import Qobj

# --- Sistem Tanımı ---
A = np.array([[1, 0.5],
              [0.5, 1]])
b = np.array([1, 0])

# |b> durumunu normalize et
b_norm = b / np.linalg.norm(b)
ket_b = Qobj(b_norm)

# --- Özdeğer Ayrışımı ---
eigvals, eigvecs = np.linalg.eigh(A)

# A^-1 işlemi
A_inv = eigvecs @ np.diag(1 / eigvals) @ eigvecs.T

# --- Çözüm Durumu ---
x_quantum = Qobj(A_inv @ b_norm).unit()

print("A:\n", A)
print("\n|b> =", b_norm)
print("\nA^-1 |b> = |x> :\n", x_quantum.full())
print("\nÖlçüm Olasılıkları:\n", np.abs(x_quantum.full())**2)


A:
 [[1.  0.5]
 [0.5 1. ]]

|b> = [1. 0.]

A^-1 |b> = |x> :
 [[ 0.89442719+0.j]
 [-0.4472136 +0.j]]

Ölçüm Olasılıkları:
 [[0.8]
 [0.2]]


Qiskit ile HHL Devresi

In [17]:
from qiskit import Aer, transpile
from qiskit.visualization import plot_histogram
from qiskit.algorithms.linear_solvers import HHL
from qiskit.algorithms.linear_solvers.observables import MatrixFunctional
from qiskit.quantum_info import Statevector
from qiskit.algorithms.linear_solvers.matrices import Matrix
from qiskit.utils import QuantumInstance
import numpy as np

# --- Sistem Tanımı ---
A = [[1, 0.5],
     [0.5, 1]]
b = [1, 0]

# --- Qiskit HHL Algoritması ---
hhl = HHL()
matrix = Matrix(A)
rhs = Statevector(b)

# --- Simülasyon ---
backend = Aer.get_backend('statevector_simulator')
result = hhl.solve(matrix, rhs, quantum_instance=QuantumInstance(backend))

# --- Çıktı ---
print("A^-1 |b> (yaklaşık |x>):")
print(result.state.solution)


ImportError: cannot import name 'Aer' from 'qiskit' (/usr/local/lib/python3.12/dist-packages/qiskit/__init__.py)

Cirq ile Basitleştirilmiş HHL Yaklaşımı

In [18]:
import cirq
import numpy as np

# Kübitler
ancilla = cirq.NamedQubit("ancilla")
q0 = cirq.NamedQubit("q0")
q1 = cirq.NamedQubit("q1")

# --- Devre tanımı ---
circuit = cirq.Circuit()

# 1. |b> durumunu hazırla (örneğin |10>)
circuit.append([cirq.X(q0)])

# 2. QPE benzeri işlem (örnek olarak küçük faz dönüşümü)
circuit.append(cirq.ry(np.pi / 4)(q1))
circuit.append(cirq.CNOT(q1, ancilla))

# 3. Kontrollü rotasyon (1/λ kodlama)
circuit.append(cirq.ry(-np.pi / 8)(ancilla))

# 4. Uncompute işlemi
circuit.append(cirq.CNOT(q1, ancilla))
circuit.append(cirq.ry(-np.pi / 4)(q1))

# 5. Ölçüm
circuit.append(cirq.measure(ancilla, q0, q1))

# --- Simülasyon ---
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1000)

print("Sonuçlar:")
print(result.histogram(key=('ancilla', 'q0', 'q1')))
print("\nDevre:\n")
print(circuit)


Sonuçlar:
Counter({2: 964, 6: 22, 7: 14})

Devre:

ancilla: ───────────────X───Ry(-0.125π)───X────────────────M───
                        │                 │                │
q0: ────────X───────────┼─────────────────┼────────────────M───
                        │                 │                │
q1: ────────Ry(0.25π)───@─────────────────@───Ry(-0.25π)───M───
